直方图

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt


# 定义一个函数，用于计算并显示灰度图像的直方图
def custom_hist_cv(gray, window_name):                               # gray：输入的灰度图像，window_name：显示窗口名称
    # cv.calcHist（）计算直方图    [gray]：输入的灰度图像（单通道） [0]：指定计算第0个通道的直方图 None：不使用掩模 [256]：直方图的bin数量为256 [0, 256]：像素值范围为0到255
    hist = cv.calcHist([gray], [0], None, [256], [0, 256])           # 返回一个256x1的数组hist，每个元素表示对应灰度级的像素数量
    # 归一化直方图数据（两个hist表示原地修改直方图数据）         采用最小-最大值归一化（cv.NORM_MINMAX） 将直方图值线性映射到[0,255]区间（alpha=0, beta=255） cv.CV_8UC1，8位无符号整型单通道
    hist = cv.normalize(hist, hist, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_8UC1)
    # 创建一个空白的黑色图像用于绘制直方图
    hist_img = np.zeros((256, 512,3), np.uint8)
    # 设置线条颜色（BGR）
    color = (255, 0, 0)                                              # 蓝色
    # 绘制直方图(线条)
    for i in range(256):
      intensity = int(hist[i][0])                                    # 表示直方图中第i个bin的统计值（如像素频数),强制转换为整数，确保后续计算精度一致（hist[i][0]：获取第i个bin的像素数量）
      # cv.line()函数用于在直方图可视化图像上绘制垂直线条       在图像的第i列绘制垂直线,起点坐标：(i, 256)（图像底部）,终点坐标：(i, 256 - intensity)（高度由频数决定，根据频数向上延伸）
      cv.line(hist_img, (i, 256), (i, 256 - intensity), color)       # 256-intensity实现垂直翻转（OpenCV坐标系原点在左上角，每个柱状图宽度为1像素（i作为x坐标）
      cv.imshow(window_name, hist_img)
    return hist_img                                                  # 用于返回包含直方图可视化结果的图像矩阵

src = cv.imread(PictureAddress)                                      # 读取图像
if src is None:
            print("Error: Image not found or path is incorrect.")
            exit()
gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)                           # 将图像从BGR颜色空间转换为灰度颜色空间
cv.namedWindow("input", cv.WINDOW_AUTOSIZE)                          # 创建可自动调整大小的窗口
cv.imshow("input", gray)                                             # 显示图像

srb = custom_hist_cv(gray, "Histogram of Orignal Image")             # 显示灰度图像的直方图

cv.waitKey(0)                                                        # 等待按键
cv.destroyAllWindows()                                               # 关闭所有窗口


直方图比较

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

src1 = cv.imread(Picture1Address)                                      # 读取图像1
src2 = cv.imread(Picture2Address)                                      # 读取图像2
#src3 = cv.imread("D:/images/flower.png")
#src4 = cv.imread("D:/images/test.jpg")
  
cv.namedWindow("input1", cv.WINDOW_AUTOSIZE)                           # 创建可自动调整大小的窗口
cv.namedWindow("input2", cv.WINDOW_AUTOSIZE)
cv.imshow("input1", src1)                                              # 显示图像1
cv.imshow("input2", src2)                                              # 显示图像2
#cv.imshow("input3", src3) 
#cv.imshow("input4", src4)
 
hsv1 = cv.cvtColor(src1, cv.COLOR_BGR2HSV)                             # BGR转换为HSV
hsv2 = cv.cvtColor(src2, cv.COLOR_BGR2HSV)
#hsv3 = cv.cvtColor(src3, cv.COLOR_BGR2HSV)
#hsv4 = cv.cvtColor(src4, cv.COLOR_BGR2HSV)

# 计算HSV色彩空间的2D直方图  [hsv1]：输入图像列表（单幅HSV图像） [0,1]：计算通道0(H)和1(S)的联合直方图 None：不使用掩模 [60,64]：H维度60个bin，S维度64个bin [0,180,0,256]：H值范围0-180，S值范围0-256
hist1 = cv.calcHist([hsv1], [0, 1], None, [60, 64], [0, 180, 0, 256])
hist2 = cv.calcHist([hsv2], [0, 1], None, [60, 64], [0, 180, 0, 256])
#hist3 = cv.calcHist([hsv3], [0, 1], None, [60, 64], [0, 180, 0, 256])
#hist4 = cv.calcHist([hsv4], [0, 1], None, [60, 64], [0, 180, 0, 256])
print(hist1.dtype)                                                      # hist1.dtype通常会输出float32或float64，因为calcHist()函数返回的直方图数据默认是浮点型数组
# 归一化
cv.normalize(hist1, hist1, 0, 1.0, cv.NORM_MINMAX, dtype=cv.CV_32F)
cv.normalize(hist2, hist2, 0, 1.0, cv.NORM_MINMAX)
#cv.normalize(hist3, hist3, 0, 1.0, cv.NORM_MINMAX)
#cv.normalize(hist4, hist4, 0, 1.0, cv.NORM_MINMAX)
 
match1= cv.compareHist(hist1, hist2, cv.HISTCMP_CORREL)                 # 计算相关性（值域[-1,1]，1表示完全匹配
match2= cv.compareHist(hist1, hist2, cv.HISTCMP_CHISQR)                 # 卡方检验（值域[0,∞)，0表示完全匹配）
match3= cv.compareHist(hist1, hist2, cv.HISTCMP_BHATTACHARYYA)          # 巴氏距离（值域[0,1]，0表示完全匹配）
match4= cv.compareHist(hist1, hist2, cv.HISTCMP_INTERSECT)              # 直方图交集（值域[0,∞)，值越大相似度越高）
print("相似性: %s，卡方: %s，巴氏距离: %s，交集: %s"%(match1, match2, match3, match4))


# 补充
#methods = [cv.HISTCMP_CORREL, cv.HISTCMP_CHISQR,cv.HISTCMP_INTERSECT, cv.HISTCMP_BHATTACHARYYA]
#str_method = ""
#for method in methods:
    #src1_src2 = cv.compareHist(hist1, hist2, method)                   # 对每对直方图(hist1/hist2和hist3/hist4)分别计算相似度
    #src3_src4 = cv.compareHist(hist3, hist4, method)
    #if method == cv.HISTCMP_CORREL:                                    # 根据方法类型动态生成可读性结果描述
    #   str_method = "Correlation"
    #if method == cv.HISTCMP_CHISQR:
    #    str_method = "Chi-square"
    #if method == cv.HISTCMP_INTERSECT:
    #    str_method = "Intersection"
    #if method == cv.HISTCMP_BHATTACHARYYA:
    #    str_method = "Bhattacharyya"
    #print("%s src1_src2 = %.2f, src3_src4 = %.2f"%(str_method, src1_src2, src3_src4))  # 数值保留2位小数

cv.waitKey(0)                                                           # 等待按键
cv.destroyAllWindows()                                                  # 关闭所有窗口

直方图均值化

In [ ]:
import cv2 as cv
import numpy as np

# 定义一个函数，用于计算并显示灰度图像的直方图
def custom_hist_cv(gray, window_name):
    # 计算直方图
    # [gray]：输入的单通道灰度图像 [0]：指定计算第0个通道（灰度图只有1个通道） None：不使用掩模图像 [256]：直方图bin的数量 [0,256]：像素值范围为0到255
    hist = cv.calcHist([gray], [0], None, [256], [0, 256])
    # 归一化直方图数据,采用最小 - 最大值归一化（cv.NORM_MINMAX)  将直方图值线性映射到[0,255]区间（alpha=0, beta=255）  cv.CV_8UC1，8位无符号整型单通道
    hist = cv.normalize(hist, hist, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_8UC1)
    # 创建一个256行×512列的三通道黑色背景图像用于绘制直方图
    hist_img = np.zeros((256, 512, 3), dtype=np.uint8)
    # 设置线条颜色（BGR
    color = (255, 0, 0)                                               # 蓝色
    # 绘制直方图
    for i in range(256):                                              # 遍历0-255的灰度级
        intensity = int(hist[i][0])                                   # 表示直方图中第i个bin的统计值（如像素频数),强制转换为整数，确保后续计算精度一致（hist[i][0]：获取第i个bin的像素数量）
        cv.line(hist_img, (i, 256), (i, 256 - intensity), color)      # 在图像的第i列绘制垂直线,起点坐标：(i, 256)（图像底部）,终点坐标：(i, 256 - intensity)（高度由频数决定，根据频数向上延伸）
        cv.imshow(window_name, hist_img)                              # 显示直方图图像
    return hist_img

src = cv.imread(PictureAddress)                                       # 读取图像文件
if src is None:
    print("Error: Image not found or path is incorrect.")
    exit()

gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)                            # 将图像从BGR颜色空间转换为灰度颜色空间
cv.namedWindow("input", cv.WINDOW_AUTOSIZE)                           # 创建可自动调整大小的窗口，cv.namedWindow("名称", " 模式")
cv.imshow("input", gray)                                              # 显示图像

# 使用直方图均衡化增强图像的对比度
dst = cv.equalizeHist(gray)                                           # （）里的输入必须是8位单通道灰度图像
# 显示直方图均衡化后的图像
cv.namedWindow("eh", cv.WINDOW_AUTOSIZE)
cv.imshow("eh", dst)

# 调用自定义函数，显示原始灰度图像的直方图
srb = custom_hist_cv(gray, "Histogram of Original Image")
# 调用自定义函数，显示直方图均衡化后的图像的直方图
srd = custom_hist_cv(dst, "Histogram of Equalized Image")

cv.waitKey(0)                                                         # 等待按键
cv.destroyAllWindows()                                                # 关闭所有窗口

直方图反向投影

In [ ]:
import cv2 as cv
import numpy as np
from  matplotlib import pyplot as plt


def back_projection_demo():                                       # 定义直方图反向投影函数
    # 读取样本图像和目标图像
    sample = cv.imread(Picture1Address)                           # 读取图像文件1
    target = cv.imread(Picture2Address)                           # 读取图像文件2
    #转换为HSV色彩空间
    roi_hsv = cv.cvtColor(sample, cv.COLOR_BGR2HSV)
    target_hsv = cv.cvtColor(target, cv.COLOR_BGR2HSV)
    # show images（显示原始图像）
    cv.namedWindow("sample", cv.WINDOW_AUTOSIZE)
    cv.namedWindow("target", cv.WINDOW_AUTOSIZE)                  # 创建可自动调整大小的窗口
    cv.imshow("sample", sample)                                   # 显示样本图像
    cv.imshow("target", target)                                   # 显示目标图像
    # 计算样本直方图（H-S通道）
    # [roi_hsv]表示输入的是单幅HSV格式图像        [0,1]指定同时计算H(色调)和S(饱和度)两个通道的联合直方图，忽略V(亮度)通道  None表示不对图像区域进行掩模处理，计算全图的直方图
    # [32,32]将H通道和S通道分别划分为32个bin，最终生成32x32的二维直方图矩阵   [0,180,0,256]指H通道和S通道的取值范围
    roiHist = cv.calcHist([roi_hsv], [0, 1], None, [32, 32], [0, 180, 0, 256])
    # 归一化直方图（cv.NORM_MINMAX采用最小-最大值归一化方法）
    # roiHist同时作为输入和输出参数，表示直方图数据将被原地修改           0, 255指定将直方图值线性缩放至0-255区间
    cv.normalize(roiHist, roiHist, 0, 255, cv.NORM_MINMAX)
    # 计算反向投影
    dst = cv.calcBackProject([target_hsv], [0, 1], roiHist, [0, 180, 0, 256], 1)
    cv.imshow("backProjectionDemo", dst)

# 计算并可视化图像的二维直方图
def hist2d_demo(image):                                           # 定义hist2d_demo()函数
    hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)                    # 转换为HSV图像
    hist = cv.calcHist([hsv], [0, 1], None, [32, 32], [0, 180, 0, 256])
    # plt.figure（）Matplotlib库中用于创建图形窗口的配置语句
    plt.figure(figsize=(10, 8), dpi=100)                          # figsize=(10,8)设置图形宽度为10英寸，高度为8英寸，控制最终输出的物理尺寸dpi=100 指定图像分辨率为100像素/英寸
    # 可视化
    # 用Matplotlib的imshow()函数显示二维直方图数据
    plt.imshow(hist, interpolation='nearest')
    
    # 补充
    #plt.title("2D Histogram")                                    # 添加标题
    #plt.colorbar()                                               # 添加色阶条
    #plt.xlabel("Saturation")                                     # 设置X轴标签
    #plt.ylabel("Hue")                                            # 设置y轴标签
    plt.show()
    return hist

back_projection_demo()                                            # 调用back_projection_demo()函数
# 读取样本图像和目标图像
sample = cv.imread(Picture1Address)
target = cv.imread(Picture2Address)
hist2d_demo(sample)                                               # 调用hist2d_demo()函数
#hist2d_demo(target)

cv.waitKey(0)                                                     # 等待按键
cv.destroyAllWindows()                                            # 关闭所有窗口